In [1]:
import pandas as pd
import os
import numpy as np
import datetime

In [2]:
#load up all the data
inputfolder = 'data'
filelist = os.listdir(inputfolder)
framedict = dict()
for file in filelist:
    filename,ext = os.path.splitext(file)
    pathway = os.path.join(inputfolder,file)
#     print(pathway)
    tempdf = pd.read_csv(pathway)
    framedict[filename] = tempdf
    del tempdf
dictnames = [x for x in framedict.keys()]
print(dictnames)

C:\Users\Hugh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,12,13,14,17,18,24,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['ATP_matches', 'aus_open_men_match_odds', 'aus_open_men_tournament_odds', 'aus_open_women_match_odds', 'aus_open_women_tournament_odds', 'cleaned_data', 'datacolumns', 'final_match_data', 'mens_match_data', 'WTA_matches']


Looking at the mens data for simplicity again

In [3]:
#load mens dataframes into variables and inspect data
ATPframe = framedict['ATP_matches'].copy() #we copy so we can run the code multiple times without errors, not really necessary
AOMensMatchOdds = framedict['aus_open_men_match_odds'].copy()
AOMensTournOdds = framedict['aus_open_men_tournament_odds'].copy()
display(ATPframe.head())
display(AOMensMatchOdds.head())
display(AOMensTournOdds.head())

#initial cleaning
#checking that we're only looking at mens data
print(AOMensMatchOdds['men_women'].unique())
print(AOMensTournOdds['men_women'].unique())

#only mens, so we can drop this column
AOMensMatchOdds.drop('men_women',axis=1,inplace=True)
AOMensTournOdds.drop('men_women',axis=1,inplace=True)

#convert dates to datetime
ATPframe['datetime'] = pd.to_datetime(ATPframe['Tournament_Date'])
AOMensMatchOdds['datetime'] = pd.to_datetime(AOMensMatchOdds['match_date'])
display(AOMensMatchOdds.loc[:,['match_date','datetime']].head())

#testing splitting up dates into day/month/year
AOMensMatchOdds['year'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).year
AOMensMatchOdds['month'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).month
AOMensMatchOdds['day'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).day
ATPframe['year'] = pd.DatetimeIndex(ATPframe['datetime']).year
ATPframe['month'] = pd.DatetimeIndex(ATPframe['datetime']).month
ATPframe['day'] = pd.DatetimeIndex(ATPframe['datetime']).day
# display(ATPframe.loc[:,['year','month','day','datetime']].head())

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,02-Jan-12,Hard,First Round,106,97,0,2,...,3,21,33,13,26,1,3,15,49,49
1,Dudi Sela,Fabio Fognini,Chennai,02-Jan-12,Hard,First Round,83,48,0,2,...,4,17,32,5,26,0,1,8,33,30
2,Go Soeda,Frederico Gil,Chennai,02-Jan-12,Hard,First Round,120,102,0,2,...,2,45,70,18,35,2,4,36,103,99
3,Yuki Bhambri,Karol Beck,Chennai,02-Jan-12,Hard,First Round,345,101,0,2,...,1,15,33,13,29,2,3,15,46,43
4,Yuichi Sugita,Olivier Rochus,Chennai,02-Jan-12,Hard,First Round,235,67,0,2,...,0,19,32,13,22,1,7,30,78,62


,tournament,men_women,match_date,player_1,player_1_odds,player_2,player_2_odds
0,Australian Open 2012,Men,2012-01-17,Potito Starace,1.77,Tatsuma Ito,2.24
1,Australian Open 2012,Men,2012-01-17,Nicolas Mahut,5.30,Radek Stepanek,1.20
2,Australian Open 2012,Men,2012-01-17,Filippo Volandri,40.00,Milos Raonic,1.02
3,Australian Open 2012,Men,2012-01-17,Lukas Rosol,3.25,Philipp Petzschner,1.42
4,Australian Open 2012,Men,2012-01-17,Cedrik Marcel Stebe,3.60,Lleyton Hewitt,1.36


,tournament,men_women,player,odds
0,Australian Open 2012,Men,Novak Djokovic,2.46
1,Australian Open 2012,Men,Roger Federer,5.60
2,Australian Open 2012,Men,Andy Murray,7.60
3,Australian Open 2012,Men,Rafael Nadal,9.00
4,Australian Open 2012,Men,Jo Wilfried Tsonga,22.00


['Men']
['Men']


,match_date,datetime
0,2012-01-17,2012-01-17
1,2012-01-17,2012-01-17
2,2012-01-17,2012-01-17
3,2012-01-17,2012-01-17
4,2012-01-17,2012-01-17


In [4]:
#"unstacking" Player data so that each row is the performance of one player in one of their matches
winframe = ATPframe.filter(regex='Winner')
loseframe = ATPframe.filter(regex='Loser')
winframe.loc[:,'won_game?'] = 1
loseframe.loc[:,'won_game?'] = 0
winlist = [x for x in winframe.columns]
loselist = [x for x in loseframe.columns]
for col in winlist[1:]:
    winframe.loc[:,col] = pd.to_numeric(winframe[col],errors="coerce")
for col in loselist[1:]:
    loseframe.loc[:,col] = pd.to_numeric(loseframe[col],errors="coerce")
colnames = [x.replace("Winner_","") if "Winner_" in x else x for x in winlist]
colnames[0] = "Player"
print(colnames)
loseframe.columns = colnames
winframe.columns = colnames
# display(loseframe[loseframe['Loser_Rank'].isna()])

#im going to calculate some ratios here because it makes sense to do so programatically, but these are explored further below
ratiolist = ['FirstServes','SecondServes','BreakPoints','ReturnPoints']
for idx,frame in enumerate([winframe,loseframe]):
    for col in ratiolist:
        tempdf = frame.filter(regex=col)
        if tempdf.shape[1] == 2:
            tempcolnames = tempdf.columns
            for col2 in tempcolnames:
                if "Won" in col2:
                    nomcol = col2
                else:
                    domcol = col2
            frame.loc[:,col+'_ratio'] =  pd.to_numeric(tempdf[nomcol],errors="coerce")/pd.to_numeric(tempdf[domcol],errors="coerce")
            frame.loc[:,col+'_ratio'].fillna(tempdf[domcol],inplace=True)
#             if frame[prefix+col+'_ratio'].isna().sum()>0:
#                 display(frame[frame[prefix+col+'_ratio'].isna()])
        else:
            raise ValueError("More than two columns matching regex: %{}".format(col))
#     print(frame.columns)
    display(frame.head())
#grabs colnames that we want to find the difference score of
winframe = winframe.merge(ATPframe.loc[:,['datetime','year','Tournament','Round_Description','Court_Surface']],left_index=True,right_index=True)
loseframe = loseframe.merge(ATPframe.loc[:,['datetime','year','Tournament','Round_Description','Court_Surface']],left_index=True,right_index=True)

display(winframe.head())
display(loseframe.head())
print("-"*200)
print("find missing values")
print(winframe.isna().sum())
print(loseframe.isna().sum())
# display(loseframe.loc[loseframe['Loser_BreakPoints_ratio'].isna(),['Loser_BreakPoints_Won','Loser_BreakPoints','Loser_BreakPoints_ratio']])

#filling unranked players with lowest rank in tournament+1 (therefore unranked players are considered the worst player in that tournament)
def fillrank(row,col,ATPframe):
    if np.isnan(row[col]):
            year = row['year']
            tournament = row['Tournament']
            surface = row['Court_Surface']
            yeardf = ATPframe[ATPframe['year']==year] #i can do this slicing in one line, but im doing in two to allow for easier debugging
            tourndf = ATPframe[(ATPframe['Tournament']==tournament)&(ATPframe['Court_Surface']==surface)]
            maxrank = int(max([tourndf['Winner_Rank'].max(),tourndf['Loser_Rank'].max()]))
            return maxrank+1
    else:
        return row[col]
print("-"*200)
print('Filling in unranked')
winframe.loc[:,'Rank'] = winframe.apply(fillrank,axis=1,args=('Rank',ATPframe))
loseframe.loc[:,'Rank'] = loseframe.apply(fillrank,axis=1,args=('Rank',ATPframe))
print(winframe['Rank'].isna().sum())
print(loseframe['Rank'].isna().sum())
display(winframe.head())
display(loseframe.head())
# display(loseframe.loc[loseframe['Loser_BreakPoints_ratio'].isna(),['Loser_BreakPoints_Won','Loser_BreakPoints','Loser_BreakPoints_ratio']])

C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


['Player', 'Rank', 'Sets_Won', 'Games_Won', 'Aces', 'DoubleFaults', 'FirstServes_Won', 'FirstServes_In', 'SecondServes_Won', 'SecondServes_In', 'BreakPoints_Won', 'BreakPoints', 'ReturnPoints_Won', 'ReturnPoints_Faced', 'TotalPoints_Won', 'won_game?']


C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,4.0,7.0,25.0,59.0,59,1,0.733333,0.631579,0.571429,0.423729
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,6.0,14.0,36.0,58.0,61,1,0.823529,0.687500,0.428571,0.620690
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,5.0,11.0,42.0,105.0,109,1,0.750000,0.487179,0.454545,0.400000
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,5.0,13.0,34.0,62.0,65,1,0.758621,0.529412,0.384615,0.548387
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,3.0,7.0,22.0,54.0,70,1,0.725490,0.407407,0.428571,0.407407


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio
0,Eric Prodon,97.0,0.0,7,2.0,3.0,21.0,33.0,13.0,26.0,1.0,3.0,15.0,49.0,49,0,0.636364,0.500000,0.333333,0.306122
1,Fabio Fognini,48.0,0.0,1,1.0,4.0,17.0,32.0,5.0,26.0,0.0,1.0,8.0,33.0,30,0,0.531250,0.192308,0.000000,0.242424
2,Frederico Gil,102.0,1.0,14,5.0,2.0,45.0,70.0,18.0,35.0,2.0,4.0,36.0,103.0,99,0,0.642857,0.514286,0.500000,0.349515
3,Karol Beck,101.0,0.0,5,1.0,1.0,15.0,33.0,13.0,29.0,2.0,3.0,15.0,46.0,43,0,0.454545,0.448276,0.666667,0.326087
4,Olivier Rochus,67.0,0.0,7,0.0,0.0,19.0,32.0,13.0,22.0,1.0,7.0,30.0,78.0,62,0,0.593750,0.590909,0.142857,0.384615


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,...,1,0.733333,0.631579,0.571429,0.423729,2012-01-02,2012,Chennai,First Round,Hard
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,...,1,0.823529,0.687500,0.428571,0.620690,2012-01-02,2012,Chennai,First Round,Hard
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,...,1,0.750000,0.487179,0.454545,0.400000,2012-01-02,2012,Chennai,First Round,Hard
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,...,1,0.758621,0.529412,0.384615,0.548387,2012-01-02,2012,Chennai,First Round,Hard
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,...,1,0.725490,0.407407,0.428571,0.407407,2012-01-02,2012,Chennai,First Round,Hard


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Eric Prodon,97.0,0.0,7,2.0,3.0,21.0,33.0,13.0,26.0,...,0,0.636364,0.500000,0.333333,0.306122,2012-01-02,2012,Chennai,First Round,Hard
1,Fabio Fognini,48.0,0.0,1,1.0,4.0,17.0,32.0,5.0,26.0,...,0,0.531250,0.192308,0.000000,0.242424,2012-01-02,2012,Chennai,First Round,Hard
2,Frederico Gil,102.0,1.0,14,5.0,2.0,45.0,70.0,18.0,35.0,...,0,0.642857,0.514286,0.500000,0.349515,2012-01-02,2012,Chennai,First Round,Hard
3,Karol Beck,101.0,0.0,5,1.0,1.0,15.0,33.0,13.0,29.0,...,0,0.454545,0.448276,0.666667,0.326087,2012-01-02,2012,Chennai,First Round,Hard
4,Olivier Rochus,67.0,0.0,7,0.0,0.0,19.0,32.0,13.0,22.0,...,0,0.593750,0.590909,0.142857,0.384615,2012-01-02,2012,Chennai,First Round,Hard


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
find missing values
Player                  0
Rank                  103
Sets_Won                9
Games_Won               0
Aces                   15
DoubleFaults           38
FirstServes_Won         1
FirstServes_In          1
SecondServes_Won        2
SecondServes_In         2
BreakPoints_Won         7
BreakPoints             7
ReturnPoints_Won      376
ReturnPoints_Faced    376
TotalPoints_Won         0
won_game?               0
FirstServes_ratio       1
SecondServes_ratio      2
BreakPoints_ratio       7
ReturnPoints_ratio    376
datetime                0
year                    0
Tournament              0
Round_Description       0
Court_Surface           0
dtype: int64
Player                  0
Rank                  300
Sets_Won                9
Games_Won               0
Aces        

,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,...,1,0.733333,0.631579,0.571429,0.423729,2012-01-02,2012,Chennai,First Round,Hard
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,...,1,0.823529,0.687500,0.428571,0.620690,2012-01-02,2012,Chennai,First Round,Hard
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,...,1,0.750000,0.487179,0.454545,0.400000,2012-01-02,2012,Chennai,First Round,Hard
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,...,1,0.758621,0.529412,0.384615,0.548387,2012-01-02,2012,Chennai,First Round,Hard
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,...,1,0.725490,0.407407,0.428571,0.407407,2012-01-02,2012,Chennai,First Round,Hard


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Eric Prodon,97.0,0.0,7,2.0,3.0,21.0,33.0,13.0,26.0,...,0,0.636364,0.500000,0.333333,0.306122,2012-01-02,2012,Chennai,First Round,Hard
1,Fabio Fognini,48.0,0.0,1,1.0,4.0,17.0,32.0,5.0,26.0,...,0,0.531250,0.192308,0.000000,0.242424,2012-01-02,2012,Chennai,First Round,Hard
2,Frederico Gil,102.0,1.0,14,5.0,2.0,45.0,70.0,18.0,35.0,...,0,0.642857,0.514286,0.500000,0.349515,2012-01-02,2012,Chennai,First Round,Hard
3,Karol Beck,101.0,0.0,5,1.0,1.0,15.0,33.0,13.0,29.0,...,0,0.454545,0.448276,0.666667,0.326087,2012-01-02,2012,Chennai,First Round,Hard
4,Olivier Rochus,67.0,0.0,7,0.0,0.0,19.0,32.0,13.0,22.0,...,0,0.593750,0.590909,0.142857,0.384615,2012-01-02,2012,Chennai,First Round,Hard


In [5]:
#concatenating my two dataframes into a single one
df = pd.concat([winframe,loseframe],axis=0,ignore_index=True)
# print((winframe.shape,loseframe.shape))
# print(df.shape)
display(df.head())
print(df.isna().sum())

#adding extra features. These came about through discussion with one my friends who has stronger domain knowledge than i do
#apparently serves are a big thing, so we need to know how consistent they are at serving, and how much of an 'advantage' each players serve gives them
df.loc[:,'Total_Serves'] = df['FirstServes_In'] + df['SecondServes_In']

#is that person particularly strong at serving
df.loc[:,'Aces%'] = df['Aces']/df['Total_Serves']
df.loc[:,'ServesWon%'] = (df['FirstServes_Won'] + df['SecondServes_Won'])/df['Total_Serves']
display(df.head())
#i'm gonna use the std dev of Aces% and ServesWon% as an indication of consistency, but these will be calculated when appending individual statistics

,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,...,1,0.733333,0.631579,0.571429,0.423729,2012-01-02,2012,Chennai,First Round,Hard
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,...,1,0.823529,0.687500,0.428571,0.620690,2012-01-02,2012,Chennai,First Round,Hard
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,...,1,0.750000,0.487179,0.454545,0.400000,2012-01-02,2012,Chennai,First Round,Hard
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,...,1,0.758621,0.529412,0.384615,0.548387,2012-01-02,2012,Chennai,First Round,Hard
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,...,1,0.725490,0.407407,0.428571,0.407407,2012-01-02,2012,Chennai,First Round,Hard


Player                  0
Rank                    0
Sets_Won               18
Games_Won               0
Aces                   32
DoubleFaults           73
FirstServes_Won         2
FirstServes_In          2
SecondServes_Won        4
SecondServes_In         4
BreakPoints_Won        22
BreakPoints            17
ReturnPoints_Won      752
ReturnPoints_Faced    752
TotalPoints_Won         0
won_game?               0
FirstServes_ratio       2
SecondServes_ratio      4
BreakPoints_ratio      17
ReturnPoints_ratio    752
datetime                0
year                    0
Tournament              0
Round_Description       0
Court_Surface           0
dtype: int64


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface,Total_Serves,Aces%,ServesWon%
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,...,0.571429,0.423729,2012-01-02,2012,Chennai,First Round,Hard,49.0,0.102041,0.693878
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,...,0.428571,0.620690,2012-01-02,2012,Chennai,First Round,Hard,33.0,0.060606,0.757576
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,...,0.454545,0.400000,2012-01-02,2012,Chennai,First Round,Hard,103.0,0.058252,0.650485
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,...,0.384615,0.548387,2012-01-02,2012,Chennai,First Round,Hard,46.0,0.021739,0.673913
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,...,0.428571,0.407407,2012-01-02,2012,Chennai,First Round,Hard,78.0,0.038462,0.615385


In [6]:
#Ok now this is going to be interesting, firstly for now, im only going to consider tournaments with the same court surface as the Aus Open (Hard)
harddf = df[df['Court_Surface']=="Hard"]
print(harddf.shape)

(18350, 28)


In [7]:
#ok so just considering hard surfaces, we have less than half of the remaining data set.
#lets see how many rows that is per year
display(harddf.groupby(['year']).agg({'year':'count'}))

#testing days_difference function to make sure it works as intended
print((datetime.datetime.now()-harddf.loc[0,'datetime']).days)
def days_difference(date1,date2):
            diff = date2-date1
            return diff.days

tempdate = datetime.datetime.now() #making a copy so we dont modify actual data
tempdf = harddf.copy()
tempdf['days_from_today'] = tempdf['datetime'].apply(days_difference,args=(tempdate,))
display(tempdf.head())
del tempdf
del tempdate
#so theres roughly 2.5k rows per year, i know this will be pretty sparse for some players, i think were ok for now
#now im going to define the function that will take a player name and return individual statistics eg. historic avg Aces etc
def get_player_stats(inputframe,harddf,datacols,playercol):
    #idea is to pass a panda dataframe with columns [playername, tournamentdate] and be able to return a dataframe with the stats
    length = inputframe.shape[0]
    historicnames = ['careeravg_'+x for x in datacols]
    ytdnames = ['ytd_'+x for x in datacols]
    colnames = [playercol,'datetime','Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev','Win%AgainstPlayer'] + historicnames+ytdnames
    outputframe = pd.DataFrame(index=range(0,length),columns=colnames)
    outputframe.loc[:,playercol] = inputframe[playercol]
    outputframe.loc[:,'datetime'] = inputframe['datetime']
    for index in range(0,length):
        #extract player name and date of game
        row = inputframe.iloc[index,:]
        playername = row[playercol]
#         otherplayername = row[otherplayercol]
        date = row['datetime']
        
        #grab only data for that player before that date
        tempdf = harddf[harddf['Player']==playername]
        tempdf.loc[:,'timedelta'] = tempdf['datetime'].apply(days_difference,args=(date,))
        tempdf = tempdf[tempdf['timedelta']>0]
        if tempdf.empty:
            continue
        ytddf = tempdf.loc[tempdf['timedelta']<=365]
#         display(ytddf.head())
        currank = tempdf.loc[tempdf['timedelta'].idxmin(),'Rank']
        historicframe = tempdf.loc[:,datacols].mean()
        historicframe.index = historicnames
        YTDframe = ytddf.loc[:,datacols].mean()
        YTDframe.index = ytdnames
        
        #code for debugging NaNs
#         if tempdf.isna().sum().sum()>0:
#             print(playername)
#             print("ytdframe shape: {}, missing values: {}".format(ytddf.shape,ytddf.isna().sum().sum()))
#             print(ytddf.isna().sum())
#             print("careerdf shape: {}, missing values: {}".format(tempdf.shape,tempdf.isna().sum().sum()))
#             print(tempdf.isna().sum())

        outputframe.loc[index,['Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev']] = [currank,ytddf['Aces%'].std(),ytddf['ServesWon%'].std()]
        outputframe.loc[index,['Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev']]
        outputframe.loc[index,historicnames]=historicframe
        outputframe.loc[index,ytdnames]=YTDframe
    return outputframe

#testing get_player_stats
testdf = harddf.loc[harddf['year']==2015,['Player','datetime']]
testdf = testdf.iloc[:10,:].reset_index(drop=True)
display(testdf.head())
datacols = harddf.drop(['Player','Rank','datetime','year','Tournament','Round_Description','Court_Surface'],axis=1).columns
print(datacols)
testoutput = get_player_stats(testdf,harddf,datacols,'Player')
display(testoutput.head())

,year
year,
2012,2412
2013,2468
2014,2542
2015,2594
2016,2768
2017,2630
2018,2560
2019,376


2571


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface,Total_Serves,Aces%,ServesWon%,days_from_today
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,...,0.423729,2012-01-02,2012,Chennai,First Round,Hard,49.0,0.102041,0.693878,2571
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,...,0.620690,2012-01-02,2012,Chennai,First Round,Hard,33.0,0.060606,0.757576,2571
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,...,0.400000,2012-01-02,2012,Chennai,First Round,Hard,103.0,0.058252,0.650485,2571
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,...,0.548387,2012-01-02,2012,Chennai,First Round,Hard,46.0,0.021739,0.673913,2571
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,...,0.407407,2012-01-02,2012,Chennai,First Round,Hard,78.0,0.038462,0.615385,2571


,Player,datetime
0,Peter Gojowczyk,2015-01-05
1,Pablo Carreno Busta,2015-01-05
2,Gilles Muller,2015-01-05
3,Guillermo Garcia-Lopez,2015-01-05
4,Elias Ymer,2015-01-05


Index(['Sets_Won', 'Games_Won', 'Aces', 'DoubleFaults', 'FirstServes_Won',
       'FirstServes_In', 'SecondServes_Won', 'SecondServes_In',
       'BreakPoints_Won', 'BreakPoints', 'ReturnPoints_Won',
       'ReturnPoints_Faced', 'TotalPoints_Won', 'won_game?',
       'FirstServes_ratio', 'SecondServes_ratio', 'BreakPoints_ratio',
       'ReturnPoints_ratio', 'Total_Serves', 'Aces%', 'ServesWon%'],
      dtype='object')


C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Player,datetime,Rank,ytd_Aces%_Stddev,ytd_ServesWon%_Stddev,Win%AgainstPlayer,careeravg_Sets_Won,careeravg_Games_Won,careeravg_Aces,careeravg_DoubleFaults,...,ytd_ReturnPoints_Faced,ytd_TotalPoints_Won,ytd_won_game?,ytd_FirstServes_ratio,ytd_SecondServes_ratio,ytd_BreakPoints_ratio,ytd_ReturnPoints_ratio,ytd_Total_Serves,ytd_Aces%,ytd_ServesWon%
0,Peter Gojowczyk,2015-01-05,122,0.0647126,0.0787427,NaN,1.71429,13.8214,7.21429,3.07143,...,76.5,80.0909,0.727273,0.809361,0.578286,0.47316,0.390158,75.2727,0.116744,0.702809
1,Pablo Carreno Busta,2015-01-05,74,0.0240188,0.042443,NaN,1.22222,13.5556,4.33333,3.11111,...,92,94.875,0.25,0.679252,0.485053,0.45115,0.339478,97.125,0.0451653,0.609778
2,Gilles Muller,2015-01-05,58,0.074688,0.0812352,NaN,1.13043,13.5217,16.1304,4,...,74.75,92.4,0.4,0.745869,0.484795,0.314286,0.344857,94.4,0.192883,0.644123
3,Guillermo Garcia-Lopez,2015-01-05,38,0.0417034,0.101647,NaN,1.15385,12.3333,4.12821,4,...,80.5,81.8,0.533333,0.719823,0.49604,0.41842,0.358626,83.4,0.0538781,0.610636
4,Elias Ymer,2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#function takes two dataframes with matching cols, optional subset col parameter, and returns a df with the differences frame1-frame2
def get_difference(frame1,frame2,colname=None, prefix1='',prefix2='',index1=None,index2=None):
    #check if index between frames are identical, if not, reset both
    idx_check = frame1.index.equals(frame2.index)
    if not(idx_check):
        frame1.reset_index(drop=True, inplace=True)
        frame2.reset_index(drop=True, inplace=True)
    if colname is None:
        colname = frame1.columns
    if frame1.shape[0] != frame2.shape[0]:
        raise ValueError('both frames must contain the same number of rows/n Frame1: %{}/mn Frame2: %{}'.format([frame1.shape[0],frame2.shape[0]]))
    tempdiffcolnames = [x+"_diff" for x in colname]
    diffcolnames = list()
    if index1 is not None:
        diffcolnames.append(index1)
    if index2 is not None:
        diffcolnames.append(index2)
    diffcolnames = diffcolnames + tempdiffcolnames
    if idx_check:
        difframe = pd.DataFrame(index=frame1.index,columns=diffcolnames,data=0)
    else:
        difframe = pd.DataFrame(index=range(0,frame1.shape[0]),columns=diffcolnames,data=0)
    if index1 is not None:
        difframe[index1] = frame1[index1]
    if index2 is not None:
        difframe[index2] = frame2[index2]
    for col in colname:
        frame1name = prefix1+col
        frame2name = prefix2+col
        difframename = col+"_diff"
        difframe[difframename]=frame1[frame1name] - frame2[frame2name]
    return difframe

#writing the function that handles taking the input frame with the columns [player1,player2,datetime] and converts it.
#will be grabbing individual data, and then calculate difference scores, then save that data to a csv file to be used for training

def convert_to_data(inputframe,matchdata,datacols,player1colname="Player1",player2colname="Player2"):
    inputframe.reset_index(drop=True,inplace=True)
    
    player1df = get_player_stats(inputframe,matchdata,datacols,player1colname)
    player2df = get_player_stats(inputframe,matchdata,datacols,player2colname)
    
    #append columns indicating whether this is a players first recorded Hard surface ATPgame (if it's their first game, Rank will return NaNs)
    player1df['Player1fg'] = player1df['Rank'].isna()
    player2df['Player2fg'] = player2df['Rank'].isna()
    
    player1df = player1df[~player1df['Player1fg']].fillna(0)
    player2df = player2df[~player2df['Player2fg']].fillna(0)
#     if player1df.isna().sum().sum() >0:
#         display(player1df)
    
    newdatacols = player1df.drop([player1colname,'datetime','Player1fg'],axis=1).columns
    outputdf = get_difference(player1df,player2df,newdatacols,index1=player1colname,index2=player2colname)
    outputdf.loc[:,'Player1fg'] = player1df['Player1fg']
    outputdf.loc[:,'Player2fg'] = player2df['Player2fg']
    outputdf.loc[:,'datetime'] = player1df['datetime']
    return outputdf

testdf = ATPframe.loc[ATPframe['year']>2015,:].reset_index(drop=True).loc[:10,['Winner','Loser','datetime']]
testdf.columns = ['Player1','Player2','datetime']
display(testdf)
tempdf = convert_to_data(testdf,harddf,datacols)
display(tempdf)

,Player1,Player2,datetime
0,Ramkumar Ramanathan,Daniel Gimeno-Traver,2016-01-04
1,John Millman,Evgeny Donskoy,2016-01-04
2,Luca Vanni,Jan-Lennard Struff,2016-01-04
3,Thomas Fabbiano,Jozef Kovalik,2016-01-04
4,Gilles Muller,Karen Khachanov,2016-01-04
5,Borna Coric,Marcel Granollers,2016-01-04
6,Ante Pavic,Nicolas Almagro,2016-01-04
7,Austin Krajicek,Rajeev Ram,2016-01-04
8,Lukas Rosol,Santiago Giraldo,2016-01-04
9,Andrey Rublev,Somdev Devvarman,2016-01-04


C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Player1,Player2,Rank_diff,ytd_Aces%_Stddev_diff,ytd_ServesWon%_Stddev_diff,Win%AgainstPlayer_diff,careeravg_Sets_Won_diff,careeravg_Games_Won_diff,careeravg_Aces_diff,careeravg_DoubleFaults_diff,...,ytd_FirstServes_ratio_diff,ytd_SecondServes_ratio_diff,ytd_BreakPoints_ratio_diff,ytd_ReturnPoints_ratio_diff,ytd_Total_Serves_diff,ytd_Aces%_diff,ytd_ServesWon%_diff,Player1fg,Player2fg,datetime
0,Ramkumar Ramanathan,Daniel Gimeno-Traver,152,-0.000630,0.007279,0,0.130435,-0.594203,1.594203,2.608696,...,0.031155,-0.030578,0.138690,-0.046535,4.000000,0.046977,-0.001755,False,False,2016-01-04
1,John Millman,Evgeny Donskoy,-63,0.006224,0.019192,0,-0.157328,-0.810345,-2.469828,0.648707,...,-0.000534,-0.038693,-0.096373,0.048034,-12.619048,-0.031430,-0.005584,False,False,2016-01-04
2,Luca Vanni,Jan-Lennard Struff,8,0.060996,0.058458,0,-0.105263,-2.482456,4.061404,-1.912281,...,-0.010908,0.049806,0.036848,-0.034763,-17.700000,0.067834,0.028339,False,False,2016-01-04
3,Thomas Fabbiano,Jozef Kovalik,-87,-0.006166,-0.050556,0,0.500000,4.916667,-2.416667,-1.333333,...,-0.017211,0.043298,0.172980,-0.268293,13.166667,-0.015641,0.046967,False,False,2016-01-04
4,Gilles Muller,Karen Khachanov,-141,0.048754,-0.047599,0,0.283019,3.138365,8.754717,0.610063,...,0.050704,-0.073109,0.232803,0.326874,17.233333,0.068038,-0.002518,False,False,2016-01-04
5,Borna Coric,Marcel Granollers,-37,0.013452,0.044346,0,-0.098148,-0.959259,-0.296154,-0.752564,...,0.018033,0.009354,-0.017712,0.036237,-5.488636,-0.011755,0.014538,False,False,2016-01-04
6,Ante Pavic,Nicolas Almagro,-14,-0.031273,-0.050723,0,-0.530612,-3.602041,-2.765306,1.846939,...,0.042176,-0.048154,0.063005,-0.323182,-14.333333,-0.026606,0.012319,False,False,2016-01-04
7,Austin Krajicek,Rajeev Ram,36,-0.003551,0.017477,0,-0.130952,0.043651,1.365079,1.488095,...,0.019260,0.006496,-0.036941,0.042785,-6.333333,-0.013902,0.008233,False,False,2016-01-04
8,Lukas Rosol,Santiago Giraldo,20,-0.000626,-0.031851,0,-0.009091,0.028788,2.734848,2.354545,...,0.081337,0.024227,-0.098597,-0.060391,9.300000,0.025517,0.032083,False,False,2016-01-04
9,Andrey Rublev,Somdev Devvarman,46,0.013015,0.018670,0,0.271552,0.202586,-0.245690,1.715517,...,0.050851,0.086610,-0.176585,-0.009052,-9.250000,0.001359,0.075044,False,False,2016-01-04


alright lets give this one a quick whirl through a basic XGBoost model, (i know i should probably look at the relationship between feature and label ala wrangleData, but im running a bit short on time and i kind of want to see how it will perform natively

In [9]:
harddf.to_csv(os.path.join(inputfolder,'mens_match_data.csv'),index=False)
with open(os.path.join(inputfolder,'datacolumns.txt'),'w') as f:
    for item in datacols:
        f.write("%s\n" % item)

In [10]:
#do the same for womens data
WTAframe = framedict['WTA_matches']
#convert dates to datetime
WTAframe['datetime'] = pd.to_datetime(WTAframe['Tournament_Date'])

#testing splitting up dates into day/month/year
WTAframe['year'] = pd.DatetimeIndex(WTAframe['datetime']).year

#"unstacking" Player data so that each row is the performance of one player in one of their matches
winframe = WTAframe.filter(regex='Winner')
loseframe = WTAframe.filter(regex='Loser')
winframe.loc[:,'won_game?'] = 1
loseframe.loc[:,'won_game?'] = 0
winlist = [x for x in winframe.columns]
loselist = [x for x in loseframe.columns]
for col in winlist[1:]:
    winframe.loc[:,col] = pd.to_numeric(winframe[col],errors="coerce")
for col in loselist[1:]:
    loseframe.loc[:,col] = pd.to_numeric(loseframe[col],errors="coerce")
colnames = [x.replace("Winner_","") if "Winner_" in x else x for x in winlist]
colnames[0] = "Player"
print(colnames)
loseframe.columns = colnames
winframe.columns = colnames
# display(loseframe[loseframe['Loser_Rank'].isna()])

#im going to calculate some ratios here because it makes sense to do so programatically, but these are explored further below
ratiolist = ['FirstServes','SecondServes','BreakPoints','ReturnPoints']
for idx,frame in enumerate([winframe,loseframe]):
    for col in ratiolist:
        tempdf = frame.filter(regex=col)
        if tempdf.shape[1] == 2:
            tempcolnames = tempdf.columns
            for col2 in tempcolnames:
                if "Won" in col2:
                    nomcol = col2
                else:
                    domcol = col2
            frame.loc[:,col+'_ratio'] =  pd.to_numeric(tempdf[nomcol],errors="coerce")/pd.to_numeric(tempdf[domcol],errors="coerce")
            frame.loc[:,col+'_ratio'].fillna(tempdf[domcol],inplace=True)
#             if frame[prefix+col+'_ratio'].isna().sum()>0:
#                 display(frame[frame[prefix+col+'_ratio'].isna()])
        else:
            raise ValueError("More than two columns matching regex: %{}".format(col))
#     print(frame.columns)
    display(frame.head())
#grabs colnames that we want to find the difference score of
winframe = winframe.merge(WTAframe.loc[:,['datetime','year','Tournament','Round_Description','Court_Surface']],left_index=True,right_index=True)
loseframe = loseframe.merge(WTAframe.loc[:,['datetime','year','Tournament','Round_Description','Court_Surface']],left_index=True,right_index=True)

display(winframe.head())
display(loseframe.head())
print("-"*200)
print("find missing values")
print(winframe.isna().sum())
print(loseframe.isna().sum())
# display(loseframe.loc[loseframe['Loser_BreakPoints_ratio'].isna(),['Loser_BreakPoints_Won','Loser_BreakPoints','Loser_BreakPoints_ratio']])

#filling unranked players with lowest rank in tournament+1 (therefore unranked players are considered the worst player in that tournament)
print("-"*200)
print('Filling in unranked')
winframe.loc[:,'Rank'] = winframe.apply(fillrank,axis=1,args=('Rank',WTAframe))
loseframe.loc[:,'Rank'] = loseframe.apply(fillrank,axis=1,args=('Rank',WTAframe))
print(winframe['Rank'].isna().sum())
print(loseframe['Rank'].isna().sum())
display(winframe.head())
display(loseframe.head())
# display(loseframe.loc[loseframe['Loser_BreakPoints_ratio'].isna(),['Loser_BreakPoints_Won','Loser_BreakPoints','Loser_BreakPoints_ratio']])

df = pd.concat([winframe,loseframe],axis=0,ignore_index=True)
df.loc[:,'Total_Serves'] = df['FirstServes_In'] + df['SecondServes_In']

df.loc[:,'Aces%'] = df['Aces']/df['Total_Serves']
df.loc[:,'ServesWon%'] = (df['FirstServes_Won'] + df['SecondServes_Won'])/df['Total_Serves']
wharddf = df[df['Court_Surface']=="Hard"]
wharddf.to_csv(os.path.join(inputfolder,'womens_match_data.csv'),index=False)

C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


['Player', 'Rank', 'Sets_Won', 'Games_Won', 'Aces', 'DoubleFaults', 'FirstServes_Won', 'FirstServes_In', 'SecondServes_Won', 'SecondServes_In', 'BreakPoints_Won', 'BreakPoints', 'ReturnPoints_Won', 'ReturnPoints_Faced', 'TotalPoints_Won', 'won_game?']


C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio
0,Johanna Larsson,84.0,2.0,13,6.0,0.0,31.0,49.0,7.0,13.0,6.0,9.0,36.0,65.0,74.0,1,0.632653,0.538462,0.666667,0.553846
1,Ana Konjuh,259.0,2.0,15,3.0,10.0,23.0,32.0,15.0,36.0,7.0,14.0,44.0,88.0,82.0,1,0.718750,0.416667,0.500000,0.500000
2,Ayumi Morita,60.0,2.0,13,3.0,2.0,33.0,46.0,9.0,22.0,4.0,4.0,31.0,64.0,73.0,1,0.717391,0.409091,1.000000,0.484375
3,Venus Williams,47.0,2.0,13,4.0,7.0,38.0,52.0,14.0,30.0,3.0,5.0,27.0,62.0,79.0,1,0.730769,0.466667,0.600000,0.435484
4,Julia Goerges,72.0,2.0,18,2.0,4.0,45.0,56.0,29.0,46.0,1.0,9.0,46.0,128.0,120.0,1,0.803571,0.630435,0.111111,0.359375


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,BreakPoints_Won,BreakPoints,ReturnPoints_Won,ReturnPoints_Faced,TotalPoints_Won,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio
0,Lourdes Dominguez-Lino,69.0,0.0,5,0.0,4.0,16.0,32.0,13.0,33.0,2.0,8.0,24.0,62.0,53.0,0,0.500000,0.393939,0.250000,0.387097
1,Roberta Vinci,14.0,1.0,12,3.0,1.0,34.0,65.0,10.0,23.0,5.0,6.0,30.0,68.0,74.0,0,0.523077,0.434783,0.833333,0.441176
2,Lucie Safarova,29.0,0.0,9,10.0,3.0,21.0,32.0,12.0,32.0,3.0,5.0,26.0,68.0,59.0,0,0.656250,0.375000,0.600000,0.382353
3,Andrea Hlavackova,134.0,0.0,9,1.0,4.0,23.0,34.0,12.0,28.0,2.0,6.0,30.0,82.0,65.0,0,0.676471,0.428571,0.333333,0.365854
4,Karin Knapp,41.0,1.0,18,6.0,4.0,52.0,70.0,30.0,58.0,2.0,2.0,28.0,102.0,110.0,0,0.742857,0.517241,1.000000,0.274510


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Johanna Larsson,84.0,2.0,13,6.0,0.0,31.0,49.0,7.0,13.0,...,1,0.632653,0.538462,0.666667,0.553846,2014-01-01,2014,Auckland,First Round,Hard
1,Ana Konjuh,259.0,2.0,15,3.0,10.0,23.0,32.0,15.0,36.0,...,1,0.718750,0.416667,0.500000,0.500000,2014-01-01,2014,Auckland,First Round,Hard
2,Ayumi Morita,60.0,2.0,13,3.0,2.0,33.0,46.0,9.0,22.0,...,1,0.717391,0.409091,1.000000,0.484375,2014-01-01,2014,Auckland,First Round,Hard
3,Venus Williams,47.0,2.0,13,4.0,7.0,38.0,52.0,14.0,30.0,...,1,0.730769,0.466667,0.600000,0.435484,2014-01-01,2014,Auckland,First Round,Hard
4,Julia Goerges,72.0,2.0,18,2.0,4.0,45.0,56.0,29.0,46.0,...,1,0.803571,0.630435,0.111111,0.359375,2014-01-01,2014,Auckland,First Round,Hard


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Lourdes Dominguez-Lino,69.0,0.0,5,0.0,4.0,16.0,32.0,13.0,33.0,...,0,0.500000,0.393939,0.250000,0.387097,2014-01-01,2014,Auckland,First Round,Hard
1,Roberta Vinci,14.0,1.0,12,3.0,1.0,34.0,65.0,10.0,23.0,...,0,0.523077,0.434783,0.833333,0.441176,2014-01-01,2014,Auckland,First Round,Hard
2,Lucie Safarova,29.0,0.0,9,10.0,3.0,21.0,32.0,12.0,32.0,...,0,0.656250,0.375000,0.600000,0.382353,2014-01-01,2014,Auckland,First Round,Hard
3,Andrea Hlavackova,134.0,0.0,9,1.0,4.0,23.0,34.0,12.0,28.0,...,0,0.676471,0.428571,0.333333,0.365854,2014-01-01,2014,Auckland,First Round,Hard
4,Karin Knapp,41.0,1.0,18,6.0,4.0,52.0,70.0,30.0,58.0,...,0,0.742857,0.517241,1.000000,0.274510,2014-01-01,2014,Auckland,First Round,Hard


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
find missing values
Player                   0
Rank                    53
Sets_Won                 4
Games_Won                0
Aces                  1665
DoubleFaults          1631
FirstServes_Won       1623
FirstServes_In        1623
SecondServes_Won      1623
SecondServes_In       1623
BreakPoints_Won       1623
BreakPoints           1623
ReturnPoints_Won      1729
ReturnPoints_Faced    1729
TotalPoints_Won       1623
won_game?                0
FirstServes_ratio     1623
SecondServes_ratio    1623
BreakPoints_ratio     1623
ReturnPoints_ratio    1729
datetime                 0
year                     0
Tournament               0
Round_Description        0
Court_Surface            0
dtype: int64
Player                   0
Rank                   160
Sets_Won                 4
Games_Won 

,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Johanna Larsson,84.0,2.0,13,6.0,0.0,31.0,49.0,7.0,13.0,...,1,0.632653,0.538462,0.666667,0.553846,2014-01-01,2014,Auckland,First Round,Hard
1,Ana Konjuh,259.0,2.0,15,3.0,10.0,23.0,32.0,15.0,36.0,...,1,0.718750,0.416667,0.500000,0.500000,2014-01-01,2014,Auckland,First Round,Hard
2,Ayumi Morita,60.0,2.0,13,3.0,2.0,33.0,46.0,9.0,22.0,...,1,0.717391,0.409091,1.000000,0.484375,2014-01-01,2014,Auckland,First Round,Hard
3,Venus Williams,47.0,2.0,13,4.0,7.0,38.0,52.0,14.0,30.0,...,1,0.730769,0.466667,0.600000,0.435484,2014-01-01,2014,Auckland,First Round,Hard
4,Julia Goerges,72.0,2.0,18,2.0,4.0,45.0,56.0,29.0,46.0,...,1,0.803571,0.630435,0.111111,0.359375,2014-01-01,2014,Auckland,First Round,Hard


,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,won_game?,FirstServes_ratio,SecondServes_ratio,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface
0,Lourdes Dominguez-Lino,69.0,0.0,5,0.0,4.0,16.0,32.0,13.0,33.0,...,0,0.500000,0.393939,0.250000,0.387097,2014-01-01,2014,Auckland,First Round,Hard
1,Roberta Vinci,14.0,1.0,12,3.0,1.0,34.0,65.0,10.0,23.0,...,0,0.523077,0.434783,0.833333,0.441176,2014-01-01,2014,Auckland,First Round,Hard
2,Lucie Safarova,29.0,0.0,9,10.0,3.0,21.0,32.0,12.0,32.0,...,0,0.656250,0.375000,0.600000,0.382353,2014-01-01,2014,Auckland,First Round,Hard
3,Andrea Hlavackova,134.0,0.0,9,1.0,4.0,23.0,34.0,12.0,28.0,...,0,0.676471,0.428571,0.333333,0.365854,2014-01-01,2014,Auckland,First Round,Hard
4,Karin Knapp,41.0,1.0,18,6.0,4.0,52.0,70.0,30.0,58.0,...,0,0.742857,0.517241,1.000000,0.274510,2014-01-01,2014,Auckland,First Round,Hard
